In [1]:
#driveへマウントする
from google.colab import drive
drive.mount('/content/drive')

#フォルダを移動する
%cd "drive/My Drive/TD3"

%matplotlib inline

!apt update
!apt install xvfb
!pip install pyvirtualdisplay

!pip install tensorboard==1.15
!pip install tb-nightly
!pip install box2d-py

%load_ext tensorboard

from pyvirtualdisplay import Display
from gym.wrappers import Monitor

import matplotlib
import numpy as np
import gym
import torch
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import pyglet

import matplotlib.animation as animation
import matplotlib.pyplot as plt

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/TD3
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/comput

In [ ]:
%matplotlib inline

from pyvirtualdisplay import Display
from gym.wrappers import Monitor

import matplotlib
import numpy as np
import gym
import torch
from TD3_Agent import Td3Agent
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import pyglet

import matplotlib.animation as animation
import matplotlib.pyplot as plt

gym.logger.set_level(40)
from collections import deque
from GymWrapper import ImgWrapper
from util import layer_init
from Noise import ActionNoise

env = None
is_image = False
if is_image:
    env = ImgWrapper(gym.make('LunarLanderContinuous-v2'), gray_scale=True)
else:
    env = gym.make('LunarLanderContinuous-v2')
#検証用にシードを固定する
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)
env.seed(10)

num_episode = 1000  # 学習エピソード数（学習に時間がかかるので短めにしています）
memory_size = 10000  # replay bufferの大きさ
initial_memory_size = 1000  # 最初に貯めるランダムな遷移の数
# ログ用の設定
episode_rewards = []
num_average_epidodes = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter(log_dir='./runs')
max_steps = env.spec.max_episode_steps  # エピソードの最大ステップ数
agent = Td3Agent(env.observation_space,
                  env.action_space,
                  device,
                  batch_size = 64,
                  memory_size=memory_size,
                  writer=writer,
                  is_image = is_image)

# 最初にreplay bufferにランダムな行動をしたときのデータを入れる
state = env.reset()
for step in range(initial_memory_size):
    action = env.action_space.sample() # ランダムに行動を選択 
    next_state, reward, done, _ = env.step(action)
    agent.memory.add(state, action, reward, done)
    state = env.reset() if done else next_state
print('%d Data collected' % (initial_memory_size))

for episode in range(num_episode):
    state = env.reset()
    episode_reward = 0
    noise = ActionNoise(env.action_space.shape[0])
    for t in range(max_steps):
        action = agent.get_action(state)  #  行動を選択
        next_state, reward, done, _ = env.step(action * env.action_space.high)
        episode_reward += reward
        agent.memory.add(state, action, reward, done)
        agent.update()  # actorとcriticを更新
        state = next_state
        if done:
            break
    episode_rewards.append(episode_reward)
    writer.add_scalar("reward", episode_reward, episode)
    print("Episode %d finished | Episode reward %f" % (episode, episode_reward))
    if episode % 20 == 0:
        sum_reward = 0.0
        for k in range(50):
            state = env.reset()
            done = False
            step = 0
            while not done and step < max_steps:
                step += 1
                action = agent.get_action(state, is_noise = False)  #  行動を選択
                next_state, reward, done, _ = env.step(action * env.action_space.high)
                sum_reward += reward
                state = next_state
        print("Episode %d finished | Average reward %f" % (episode, sum_reward / 50))

# 累積報酬の移動平均を表示
moving_average = np.convolve(episode_rewards, np.ones(num_average_epidodes)/num_average_epidodes, mode='valid')
plt.plot(np.arange(len(moving_average)),moving_average)
plt.title('TD3: average rewards in %d episodes' % num_average_epidodes)
plt.xlabel('episode')
plt.ylabel('rewards')
plt.show()

env.close()

1000 Data collected
Episode 0 finished | Episode reward -108.809824
Episode 0 finished | Average reward -150.106260
Episode 1 finished | Episode reward -159.184909
Episode 2 finished | Episode reward -342.569327
Episode 3 finished | Episode reward -120.344378
Episode 4 finished | Episode reward -98.171654
Episode 5 finished | Episode reward -143.010013
Episode 6 finished | Episode reward -92.465577
Episode 7 finished | Episode reward -203.999167
Episode 8 finished | Episode reward -189.200015
Episode 9 finished | Episode reward -164.235236
Episode 10 finished | Episode reward -170.064386
Episode 11 finished | Episode reward -337.922111
Episode 12 finished | Episode reward -323.629694
Episode 13 finished | Episode reward -202.684997
Episode 14 finished | Episode reward -267.396534
Episode 15 finished | Episode reward -222.231390
Episode 16 finished | Episode reward -85.942572
Episode 17 finished | Episode reward 12.667164
Episode 18 finished | Episode reward -243.627137
Episode 19 finis